In [1]:
import pandas as pd
import math


In [2]:
# Read the CSV file into a DataFrame
dams_df = pd.read_csv('../input/bc-hydro-dam-coordinates/bc-hydro-dams-as-of-2024-01-01.csv')

# Dams bounding box buffer in kilometers
buffer_km = 100


In [3]:
# Function to convert DMS to DD
def dms_to_dd(dms_str):
    parts = dms_str.split('°')
    degrees = float(parts[0])
    minutes, seconds_direction = parts[1].split('′')
    seconds, direction = seconds_direction.split('″')
    dd = float(degrees) + float(minutes) / 60 + float(seconds) / 3600
    if direction == 'S' or direction == 'W':
        dd *= -1
    return dd

# Function to calculate the buffer in degrees from kilometers
def buffer_in_degrees(lat, km):
    # One degree of latitude is approximately 111 km
    buffer_lat = km / 111
    
    # Calculate the buffer in degrees for longitude
    buffer_lon = km / (111 * math.cos(math.radians(lat)))
    
    return buffer_lat, buffer_lon

# Convert the 'Location' column to decimal degrees
dams_df['Latitude'], dams_df['Longitude'] = zip(*dams_df['Location'].apply(lambda loc: [dms_to_dd(part) for part in loc.split()]))

# Find the extreme coordinates
min_lat, max_lat = dams_df['Latitude'].min(), dams_df['Latitude'].max()
min_lon, max_lon = dams_df['Longitude'].min(), dams_df['Longitude'].max()

# Calculate the buffer in degrees
buffer_lat, buffer_lon  = buffer_in_degrees((min_lat + max_lat) / 2, buffer_km)

# Apply the buffer
min_lat -= buffer_lat
min_lon -= buffer_lon
max_lat += buffer_lat
max_lon += buffer_lon

# The resulting bounding box
# Note that the Earthdata (GES DISC) dataset download form requires a precision of 3 decimal places and this particular ordering
# when selecting a subset of the data to be download.
bounding_box = [round(min_lon, 3), round(min_lat, 3), round(max_lon, 3), round(max_lat, 3)]
print(f"Bounding Box with {buffer_km}km buffer: {bounding_box}")

Bounding Box with 100km buffer: [-131.206, 47.576, -113.633, 56.917]
